The 4th experiment with fat model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
from torch.utils.data import DataLoader

from flow_regressors.flow_dataset import FlowDataset
from flow_regressors.regressors import DensePoseRegressorV7

from training.train_flow import train
from training.loss_pose import LossPose

from tqdm.auto import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Data

In [4]:
train_data = FlowDataset(
    root_path='/home/project/data_sample/flow/train',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

train_loader = DataLoader(train_data, batch_size=96, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = FlowDataset(
    root_path='/home/project/data_sample/flow/val',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

val_loader = DataLoader(val_data, batch_size=96, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

### 2. Configuration

In [5]:
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

config = dict(
    general = dict(
        experiment_name='FLOW_TO_POSE_EXP_13',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        
        n_epochs=10,
        n_steps_per_epoch=len(train_loader.dataset) // train_loader.batch_size,
        n_accum_steps=1,
        batch_size=train_loader.batch_size,
        
        swa=False,
        n_epochs_swa=None,
        n_steps_between_swa_updates=None,
          
        repeat_val_epoch=1,
        repeat_save_epoch=5,
          
        model_save_path='../src/weights/FLOW_TO_POSE_EXP_13'
    )
)

### 3. Model

In [6]:
# Inference on (120, 160) input:

model = DensePoseRegressorV7(init_loss_weights=[0.0, -3.0, 0.0])
model.to(config['general']['device']);

In [7]:
# data = train_data[0]
# data['flow'] = torch.from_numpy(data['flow']).cuda().unsqueeze(0) 
# data['K_q'] = torch.from_numpy(data['K_q']).cuda().unsqueeze(0) 
# data['K_s'] = torch.from_numpy(data['K_s']).cuda().unsqueeze(0) 
# model.eval();

In [8]:
# %%timeit

# with torch.no_grad():
#     q, t = model(
#         data['flow'], data['K_q'], data['K_s'], (0.25, 0.25), (0.25, 0.25)
#        )

### 4. Loss, optimizer, scheduler

In [9]:
train_loss = LossPose(agg_type='mean')
val_loss = LossPose()

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
config['optimizer'] = optimizer.__dict__['defaults']

In [11]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                   step_size=2,
                   gamma=0.8)

### 6. Experiment

In [12]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config['general'])

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [05:44<00:00,  1.29it/s]


epoch 0: train loss=-9.709765609830738, val loss(q)=0.10286514050810616, val loss(t)=0.45860427408551213


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [05:44<00:00,  1.29it/s]


epoch 1: train loss=-11.414719496000034, val loss(q)=0.09130869696219748, val loss(t)=0.4150731075763449


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [05:44<00:00,  1.29it/s]


epoch 8: train loss=-16.167772798094546, val loss(q)=0.06461971895874337, val loss(t)=0.32228138759087954


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 444/444 [05:44<00:00,  1.29it/s]

epoch 9: train loss=-16.54327456149026, val loss(q)=0.06411263696492553, val loss(t)=0.3198985508857355


Train batch loss,█▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
Train loss epoch,█▆▅▄▄▃▂▂▁▁
Val loss epoch(q),█▆▄▃▃▂▂▂▁▁
Val loss epoch(t),█▆▄▄▃▂▂▂▁▁
Train batch loss,-16.86061
Train loss epoch,-16.54327
Val loss epoch(q),0.06411
Val loss epoch(t),0.3199
